Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
# ## DONT CHANGE THIS CELL 
# # this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

In [ ]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=xxxxxxxxxxx

In [8]:
import os
import pandas
import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer

In [31]:
x_test = pandas.read_csv("data/test.data.txt", delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
word1 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
word2 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
x_test = x_test.drop("position", axis=1)
x_test.loc[:, "POS"] = x_test.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
x_test.insert(3, "word1", word1)
x_test.insert(5, "word2", word2)
x_test.to_csv("data/test.csv", header=None, index=False)

In [32]:
import dill
with open(os.path.join("data", "Field_TEXT"), 'rb') as f:
    TEXT = dill.load(f)

In [33]:
TEXT.vocab.vectors.shape

torch.Size([7462, 100])

In [36]:
fields = [
    ('word', TEXT),
    ('POS', data.Field(use_vocab=False, sequential=False)), 
    ('sen1', TEXT),
    ('word1', data.Field(use_vocab=False, sequential=False)),
    ('sen2', TEXT),
    ('word2', data.Field(use_vocab=False, sequential=False)),
]

test_set = data.TabularDataset.splits(
    path = "data",
    test = "test.csv",
    format = 'csv',
    fields = fields,
    skip_header = False
)

(test_itr,) = data.BucketIterator.splits(
    test_set,
    #sort_key = lambda sample : len(sample.sen1),
    sort = False,
    batch_size = 32,
    repeat = False
)

In [37]:
test_itr

In [46]:
pred = np.(pandas.read_csv("output.txt", header=None)[0])
gold = list(pandas.read_csv("data/test.gold.txt", header=None)[0])

In [45]:
f = open('output.txt', 'r').readlines()
f

['T\n', 'F\n', 'T\n']

In [ ]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
    prediciton, gold = prediction.strip(), gold.strip()
    total += 1
    if prediction == gold:
        correct += 1

## Report this as the final validation performance 
print('Performance = '%(correct/total))